In [1]:
import sys
import os
sys.path.append('./yolov3')

In [2]:
from pycocotools.coco import COCO
import numpy as np
import cv2

import albumentations as A
from albumentations.pytorch.transforms import ToTensorV2

import yaml
import torch
from models.yolo import Model
from pycocotools.cocoeval import COCOeval
import pandas as pd

from torch.utils.data import DataLoader, Dataset
import torchvision
from utils.general import coco80_to_coco91_class, check_dataset, check_file, check_img_size, check_requirements, \
    box_iou, non_max_suppression, scale_coords, xyxy2xywh, xywh2xyxy, set_logging, increment_path, colorstr
from tqdm import tqdm

In [3]:
class CustomDataset(Dataset):
    '''
      data_dir: data가 존재하는 폴더 경로
      transforms: data transform (resize, crop, Totensor, etc,,,)
    '''

    def __init__(self, annotation, data_dir, transforms=None, image_size=512):
        super().__init__()
        self.data_dir = data_dir
        self.image_size = image_size
        self.coco = COCO(annotation)
        self.predictions = {
            "images": self.coco.dataset["images"].copy(),
            "categories": self.coco.dataset["categories"].copy(),
            "annotations": None
        }
        self.transforms = transforms

    def __getitem__(self, index: int):
        image_id = self.coco.getImgIds(imgIds=index)
        image_info = self.coco.loadImgs(image_id)[0]
        image = cv2.imread(os.path.join(self.data_dir, image_info['file_name']))
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB).astype(np.float32)
        image /= 255.0

        ann_ids = self.coco.getAnnIds(imgIds=image_info['id'])
        anns = self.coco.loadAnns(ann_ids)

        # boxes (x, y, w, h)
        boxes = np.array([x['bbox'] for x in anns])
        # boxes (x_min, x_max, y_min, y_max)
        boxes[:, 2] = (boxes[:, 0] + boxes[:, 2]) 
        boxes[:, 3] = (boxes[:, 1] + boxes[:, 3])
        boxes[:, 0] /= int(self.image_size)
        boxes[:, 1] /= int(self.image_size)
        boxes[:, 2] /= int(self.image_size)
        boxes[:, 3] /= int(self.image_size)
        
        labels = list([x['category_id'] for x in anns])
        
        areas = np.array([x['area'] for x in anns])
        areas = torch.as_tensor(areas, dtype=torch.float32)
        
        is_crowds = np.array([x['iscrowd'] for x in anns])
        is_crowds = torch.as_tensor(is_crowds, dtype=torch.int64)
        
        segmentation = np.array([x['segmentation'] for x in anns], dtype=object)

        
        target = {'boxes': boxes, 'labels': labels, 'image_id': torch.tensor([index]), 'area': areas,
                  'iscrowd': is_crowds}

        # transform
        if self.transforms:
            sample = {
                'image': image,
                'bboxes': target['boxes'],
                'labels': labels
            }
            sample = self.transforms(**sample)
            image = sample['image']
            target['boxes'] = [list(box) for box in sample['bboxes']]
            

        result = []
        labels_out = torch.zeros((len(labels), 6))
        for i in range(len(labels)):
            result.append(
                [labels[i], target['boxes'][i][0], target['boxes'][i][1], target['boxes'][i][2], target['boxes'][i][3]])
        result = np.array(result)

        if len(labels):
            labels_out[:, 1:] = torch.from_numpy(result)

        return image, labels_out, image_id
    
    def __len__(self) -> int:
        return len(self.coco.getImgIds())

In [4]:
def get_train_transform():
    return A.Compose([
        A.Resize(512, 512),
        A.Flip(p=0.5),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})


def get_valid_transform(image_size):
    return A.Compose([
        A.Resize(image_size, image_size),
        ToTensorV2(p=1.0)
    ], bbox_params={'format': 'pascal_voc', 'label_fields': ['labels']})

In [5]:
class Averager:
    def __init__(self):
        self.current_total = 0.0
        self.iterations = 0.0

    def send(self, value):
        self.current_total += value
        self.iterations += 1

    @property
    def value(self):
        if self.iterations == 0:
            return 0
        else:
            return 1.0 * self.current_total / self.iterations

    def reset(self):
        self.current_total = 0.0
        self.iterations = 0.0


def collate_fn(batch):
    img, label, image_id = zip(*batch)  # transposed
    for i, l in enumerate(label):
        l[:, 0] = i  # add target image index for build_targets()
    return torch.stack(img, 0), torch.cat(label, 0), image_id


In [6]:
def valid_fn(val_data_loader, model, device, conf_thres=0.001, iou_thres=0.01):
    outputs = []
    for images, targets, image_ids in tqdm(val_data_loader):
        # gpu 계산을 위해 image.to(device)
        output, _ = model(images.to(device))
        targets = targets.to(device)
        nb, _, height, width = images.shape  # batch size, channels, height, width
        # Run NMS
        targets[:, 2:] *= torch.Tensor([width, height, width, height]).to(device)  # to pixels
        lb = [targets[targets[:, 0] == i, 1:] for i in range(nb)]  # for autolabelling
        output = non_max_suppression(output, conf_thres=conf_thres, iou_thres=iou_thres, labels=lb)
        for out in output:
            outputs.append({'boxes': out[:, 0:4].data.cpu().numpy(),
                            'scores': out[:, 4].data.cpu().numpy(), 
                            'labels': out[:, 5].data.cpu().numpy()})

    return outputs

In [7]:
def main():
    data_dir = '../input/data'
    annotation = '../input/data/train.json'
    image_size = 512
    val_dataset = CustomDataset(annotation, data_dir, get_valid_transform(image_size))
    score_threshold = 0.1
    check_point = './yolov3/weights/epoch_1.pth'

    val_data_loader = DataLoader(
        val_dataset,
        batch_size=8,
        shuffle=False,
        num_workers=4,
        collate_fn=collate_fn
    )
    device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')
    print(device)

    num_classes = 11  # 1 class (wheat) + background

    model = Model(cfg='./yolov3/models/yolov3.yaml', ch=3, nc=num_classes)  # create

    # get number of input features for the classifier
    model.to(device)

    # Hyperparameters
    with open('./yolov3/data/hyp.scratch.yaml') as f:
        hyp = yaml.load(f, Loader=yaml.SafeLoader)  # load hyps
    model.hyp = hyp
    
    model.load_state_dict(torch.load(check_point))
    model.eval()
    outputs = valid_fn(val_data_loader, model, device)
    
    prediction_strings = []
    file_names = []
    coco = COCO(annotation)
    for i, output in enumerate(outputs):
        prediction_string = ''
        image_info = coco.loadImgs(coco.getImgIds(imgIds=i))[0]
        for box, score, label in zip(output['boxes'], output['scores'], output['labels']):
            prediction_string += str(int(label)) + ' ' + str(score) + ' ' + str(box[0]) + ' ' + str(
                box[1]) + ' ' + str(box[2]) + ' ' + str(box[3]) + ' '
        prediction_strings.append(prediction_string)
        file_names.append(image_info['file_name'])
        
    submission = pd.DataFrame()
    submission['PredictionString'] = prediction_strings
    submission['image_id'] = file_names
    submission.to_csv(f'submission.csv', index=None)
    print(submission.head())

In [8]:
if __name__ == '__main__':
    main()

loading annotations into memory...
Done (t=3.84s)
creating index...
index created!
cuda


100%|██████████| 328/328 [01:03<00:00,  5.18it/s]


loading annotations into memory...
Done (t=4.25s)
creating index...
index created!
                                    PredictionString              image_id
0  1 1.0 94.899994 76.549995 358.5 290.25 8 1.0 1...  batch_01_vt/0003.jpg
1  6 1.0 -169.75 48.449997 313.95 345.15002 2 0.0...  batch_01_vt/0005.jpg
2  6 1.0 -73.600006 -27.84999 339.0 390.05 4 1.0 ...  batch_01_vt/0006.jpg
3  6 1.0 68.69998 -21.25 510.9 165.85 2 0.0021363...  batch_01_vt/0007.jpg
4  1 1.0 -102.15 -46.199997 234.35 325.40002 2 0....  batch_01_vt/0010.jpg
